In [1]:
import numpy as np

# --- 1. MULTI-AGENT ENVIRONMENT ---
class MultiAgentGridWorld:
    def __init__(self):
        self.grid_size = 4
        self.actions = [0, 1, 2, 3] # UP, DOWN, LEFT, RIGHT

        # Goals
        self.goal_A = (3, 3) # Bottom-Right
        self.goal_B = (0, 3) # Top-Right

    def reset(self):
        # Start positions
        self.pos_A = (0, 0)
        self.pos_B = (3, 0)

        # Return combined state: (rowA, colA, rowB, colB)
        return self.pos_A + self.pos_B

    def step(self, action_A, action_B):
        # 1. Calculate Proposed New Positions
        new_pos_A = self._move(self.pos_A, action_A)
        new_pos_B = self._move(self.pos_B, action_B)

        reward_A = -1
        reward_B = -1
        done_A = False
        done_B = False

        # 2. Check for Collisions (Agents hitting each other)
        if new_pos_A == new_pos_B:
            # Crash! Both stay in place and get big penalty
            reward_A = -10
            reward_B = -10
            new_pos_A = self.pos_A
            new_pos_B = self.pos_B
        else:
            # 3. Check for Goals
            if new_pos_A == self.goal_A:
                reward_A = 100
                done_A = True

            if new_pos_B == self.goal_B:
                reward_B = 100
                done_B = True

        # Update positions (if not done)
        if not done_A: self.pos_A = new_pos_A
        if not done_B: self.pos_B = new_pos_B

        next_state = self.pos_A + self.pos_B

        # Global Done: When BOTH finished
        # (For simplicity in this simulation, we reset if ONE finishes to keep them training together,
        # or we could wait. Here we'll treat episode as done if EITHER finishes for faster training cycles)
        done = done_A or done_B

        return next_state, reward_A, reward_B, done

    def _move(self, pos, action):
        r, c = pos
        if action == 0:   r = max(r - 1, 0) # UP
        elif action == 1: r = min(r + 1, self.grid_size - 1) # DOWN
        elif action == 2: c = max(c - 1, 0) # LEFT
        elif action == 3: c = min(c + 1, self.grid_size - 1) # RIGHT
        return (r, c)

# --- 2. INDEPENDENT Q-LEARNING ---
def train_marl():
    env = MultiAgentGridWorld()

    # State Space: 4x4 for Agent A * 4x4 for Agent B = 256 states
    # We map state tuple (r1, c1, r2, c2) to an index 0-255
    def get_state_idx(state_tuple):
        r1, c1, r2, c2 = state_tuple
        # Flattening 4D coordinate to 1D index
        return r1*64 + c1*16 + r2*4 + c2

    # Two Independent Q-Tables
    Q_A = np.zeros((256, 4))
    Q_B = np.zeros((256, 4))

    # Hyperparameters
    episodes = 5000
    alpha = 0.1
    gamma = 0.95
    epsilon = 0.1

    print("Training Multi-Agent System (Agents A & B)...")

    for episode in range(episodes):
        state = env.reset()
        state_idx = get_state_idx(state)
        done = False

        while not done:
            # --- ACTION SELECTION (Epsilon-Greedy) ---
            if np.random.rand() < epsilon:
                act_A = np.random.choice(env.actions)
            else:
                act_A = np.argmax(Q_A[state_idx])

            if np.random.rand() < epsilon:
                act_B = np.random.choice(env.actions)
            else:
                act_B = np.argmax(Q_B[state_idx])

            # --- STEP ---
            next_state, rA, rB, done = env.step(act_A, act_B)
            next_state_idx = get_state_idx(next_state)

            # --- UPDATE Q-TABLES SEPARATELY ---

            # Update Agent A
            old_val_A = Q_A[state_idx, act_A]
            next_max_A = np.max(Q_A[next_state_idx])
            Q_A[state_idx, act_A] = old_val_A + alpha * (rA + gamma * next_max_A - old_val_A)

            # Update Agent B
            old_val_B = Q_B[state_idx, act_B]
            next_max_B = np.max(Q_B[next_state_idx])
            Q_B[state_idx, act_B] = old_val_B + alpha * (rB + gamma * next_max_B - old_val_B)

            state_idx = next_state_idx

            # Safety break
            if rA == 100 or rB == 100:
                break

    return Q_A, Q_B, env

# --- 3. ANALYSE / TEST ---
if __name__ == "__main__":
    qa, qb, env = train_marl()

    print("\n--- Testing MARL Interaction ---")
    print("Agent A: (0,0) -> (3,3)")
    print("Agent B: (3,0) -> (0,3)")

    state = env.reset()
    state_idx = 0 # Calculated manually for (0,0,3,0)

    # Helper to print grid
    def print_grid(pos_a, pos_b):
        grid = [[' . ' for _ in range(4)] for _ in range(4)]
        grid[pos_a[0]][pos_a[1]] = ' A '
        grid[pos_b[0]][pos_b[1]] = ' B '
        if pos_a == pos_b: grid[pos_a[0]][pos_a[1]] = ' X ' # Collision
        for row in grid:
            print("".join(row))
        print("-" * 12)

    pos_A = (0,0)
    pos_B = (3,0)

    print_grid(pos_A, pos_B)

    for step in range(8):
        # Calculate state index from current positions
        idx = pos_A[0]*64 + pos_A[1]*16 + pos_B[0]*4 + pos_B[1]

        # Choose best actions
        act_A = np.argmax(qa[idx])
        act_B = np.argmax(qb[idx])

        move_map = {0:'UP', 1:'DOWN', 2:'LEFT', 3:'RIGHT'}
        print(f"Step {step+1}: Agent A goes {move_map[act_A]}, Agent B goes {move_map[act_B]}")

        # Execute (using internal env logic manually to show steps)
        # Note: In test, we assume they learned to avoid collision
        new_state, _, _, _ = env.step(act_A, act_B)

        # Extract positions from tuple (rA, cA, rB, cB)
        pos_A = (new_state[0], new_state[1])
        pos_B = (new_state[2], new_state[3])

        print_grid(pos_A, pos_B)

        if pos_A == (3,3) and pos_B == (0,3):
            print("Both Agents Reached Goals!")
            break

Training Multi-Agent System (Agents A & B)...

--- Testing MARL Interaction ---
Agent A: (0,0) -> (3,3)
Agent B: (3,0) -> (0,3)
 A  .  .  . 
 .  .  .  . 
 .  .  .  . 
 B  .  .  . 
------------
Step 1: Agent A goes RIGHT, Agent B goes RIGHT
 .  A  .  . 
 .  .  .  . 
 .  .  .  . 
 .  B  .  . 
------------
Step 2: Agent A goes DOWN, Agent B goes UP
 .  .  .  . 
 .  A  .  . 
 .  B  .  . 
 .  .  .  . 
------------
Step 3: Agent A goes RIGHT, Agent B goes RIGHT
 .  .  .  . 
 .  .  A  . 
 .  .  B  . 
 .  .  .  . 
------------
Step 4: Agent A goes RIGHT, Agent B goes UP
 .  .  .  . 
 .  .  B  A 
 .  .  .  . 
 .  .  .  . 
------------
Step 5: Agent A goes DOWN, Agent B goes UP
 .  .  B  . 
 .  .  .  . 
 .  .  .  A 
 .  .  .  . 
------------
Step 6: Agent A goes DOWN, Agent B goes RIGHT
 .  .  B  . 
 .  .  .  . 
 .  .  .  A 
 .  .  .  . 
------------
Step 7: Agent A goes DOWN, Agent B goes RIGHT
 .  .  B  . 
 .  .  .  . 
 .  .  .  A 
 .  .  .  . 
------------
Step 8: Agent A goes DOWN, Agent B g

### Understanding the Multi-Agent Reinforcement Learning Code

This notebook implements a basic Multi-Agent Reinforcement Learning (MARL) system using Independent Q-Learning. The goal is to train two agents to navigate a 4x4 grid world to their respective goals while avoiding collisions.

#### 1. MultiAgentGridWorld Environment

This class defines the grid world environment where two agents, A and B, operate.

*   **`__init__(self)`**: Initializes the grid size (4x4) and defines the possible actions (UP, DOWN, LEFT, RIGHT). It also sets the distinct goal locations for Agent A `(3, 3)` and Agent B `(0, 3)`.

*   **`reset(self)`**: Resets the environment to its initial state. Both agents start at predefined positions: Agent A at `(0, 0)` (top-left) and Agent B at `(3, 0)` (bottom-left). It returns a combined state tuple `(rowA, colA, rowB, colB)`.

*   **`step(self, action_A, action_B)`**: This is the core interaction function. Given actions for both agents, it calculates their new proposed positions.
    *   **Collision Detection**: If both agents attempt to move to the same cell, a collision occurs. In this case, both agents receive a large negative reward (`-10`), and they remain in their original positions (they don't move).
    *   **Goal Achievement**: If an agent reaches its designated goal, it receives a large positive reward (`100`), and its `done` flag is set to `True`.
    *   **Reward**: A small negative reward (`-1`) is given at each step to encourage faster goal achievement.
    *   **State Update**: The agents' positions are updated.
    *   **Global `done`**: The episode is considered `done` if *either* agent reaches its goal, simplifying the training cycles.

*   **`_move(self, pos, action)`**: A helper private method to calculate a new position for a single agent based on its current position and chosen action, ensuring it stays within grid boundaries.

#### 2. Independent Q-Learning Implementation (`train_marl` function)

This function implements the training logic for the two agents using Independent Q-Learning. In this approach, each agent learns its own Q-table independently, treating the other agent as part of the environment.

*   **`env = MultiAgentGridWorld()`**: Creates an instance of our environment.

*   **State Representation**:
    *   The environment's state is a tuple `(r1, c1, r2, c2)`.
    *   `get_state_idx(state_tuple)`: This function converts the 4-element state tuple into a single integer index (0-255). This is crucial for indexing the Q-tables, as a 4x4 grid for two agents means 4*4*4*4 = 256 possible combined states.

*   **Q-Tables (`Q_A`, `Q_B`)**: Two separate Q-tables are maintained, one for each agent. Each table has dimensions `(number_of_states, number_of_actions)`. `np.zeros((256, 4))` initializes these tables with zeros.

*   **Hyperparameters**:
    *   `episodes = 5000`: Number of training episodes.
    *   `alpha = 0.1`: Learning rate, determining how much new information overrides old information.
    *   `gamma = 0.95`: Discount factor, balancing the importance of immediate versus future rewards.
    *   `epsilon = 0.1`: Exploration-exploitation trade-off parameter. During training, agents will sometimes take random actions (`epsilon` chance) to explore the environment, and sometimes take the best-known action (`1-epsilon` chance) to exploit what they've learned.

*   **Training Loop**:
    *   For each episode, the environment is reset.
    *   **Action Selection (Epsilon-Greedy)**: Each agent independently decides its action. With probability `epsilon`, it chooses a random action. Otherwise, it chooses the action with the highest Q-value for its current state from its Q-table.
    *   **`env.step(act_A, act_B)`**: Both agents' chosen actions are passed to the environment to get the next state, rewards, and `done` flag.
    *   **Q-Table Update**: The core of Q-learning. For each agent, its Q-table is updated using the Q-learning formula:
        `Q(s,a) = Q(s,a) + alpha * (reward + gamma * max(Q(s',a')) - Q(s,a))`
        This updates the Q-value for the `(state, action)` pair based on the received reward and the maximum possible future reward from the `next_state`.
    *   The loop continues until the episode is `done` (an agent reaches its goal). A safety break is also included if a goal is reached.

#### 3. Analysis / Testing (`if __name__ == "__main__":` block)

This section runs after the training is complete to demonstrate the learned policies of the agents. It shows how the agents behave in the environment using their trained Q-tables.

*   **`qa, qb, env = train_marl()`**: Calls the training function to get the trained Q-tables and a fresh environment instance.

*   **`print_grid(pos_a, pos_b)`**: A helper function to visualize the agents' positions on the grid. 'A' represents Agent A, 'B' represents Agent B, and 'X' indicates a collision.

*   **Testing Loop**:
    *   The environment is reset to the initial state.
    *   In each step, the `get_state_idx` function is used to convert the current positions into a state index.
    *   **Action Selection**: Unlike training, during testing, agents *always* choose the action with the highest Q-value (greedy policy) from their respective Q-tables, as `epsilon` is effectively 0.
    *   **`env.step(act_A, act_B)`**: The chosen actions are executed in the environment.
    *   The new positions are extracted and the grid is printed.
    *   The loop continues for a fixed number of steps (`8` in this case) or until both agents reach their goals, demonstrating the learned cooperative/non-colliding behavior.